This is a temporary and incomplete set of tests.

In [1]:
from qiskit import *
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
from GraphMitigation import pairwise_mitigation_circuits, PairwiseMitigationFitter

In [2]:
N = 10

qc = QuantumCircuit(N)

strings = []
for circuit in pairwise_mitigation_circuits(qc):
    backend = AerSimulator()
    job = backend.run(circuit, shots=1, memory=True)
    result = job.result()
    strings.append(result.get_memory()[0])
     
for j in range(N-1):
    for k in range(j+1,N):
        substrings = []
        for string in strings:
            substring = string[j]+string[k]
            if substring not in substrings:
                substrings.append(substring)
        for substring in ['00','01','10','11']:
            assert(substring in substrings)

In [3]:
from qiskit_aer.noise import NoiseModel, depolarizing_error, thermal_relaxation_error, ReadoutError, pauli_error
from qiskit_aer import AerSimulator
def get_noise(p):

    error_meas = pauli_error([('X',p), ('I', 1 - p)])

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure") # measurement error is applied to measurements
        
    return noise_model

In [4]:
circuits = pairwise_mitigation_circuits(qc)
backend = AerSimulator()
noise_model = get_noise(0.1)
job = backend.run(circuits, noise_model=noise_model)
results = job.result()
fit = PairwiseMitigationFitter(results, circuits)
fit.mitigate_counts({'00':100,'01':10,'10':10},(0,1))

{'00': np.float64(120.0),
 '01': np.float64(0.0),
 '10': np.float64(0.0),
 '11': np.float64(0.0)}